In [1]:
import pandas as pd

"""
This notebook requires a call_of_duty_matches.csv file to be present in the project's data folder.
Each row of the csv should represent a call of duty match.
The csv should contain 3 columns:
Shots: The number of shots you fired in the match
Movement: The percentage of time you spent moving in the match (number between 0 and 1 inclusive)
KD_Spread: The number of kills you scored in the match minus the number of times you died
"""

df = pd.read_csv('data/call_of_duty_matches.csv')
df['Shots_Scaled'] = df['Shots'] / df['Shots'].max()
df['KD_Spread_Scaled'] = (df['KD_Spread'] - df['KD_Spread'].mean()) / df['KD_Spread'].std()
df.head()

,Shots,Movement,KD_Spread,Shots_Scaled,KD_Spread_Scaled
0,313,0.8848,-6,0.240215,-0.661264
1,102,0.9831,-17,0.078281,-2.427905
2,165,0.7911,-9,0.126631,-1.143076
3,591,0.7821,-4,0.453569,-0.340057
4,313,1.0000,0,0.240215,0.302358


In [2]:
from sklearn.model_selection import train_test_split
from my_micrograd.engine import Value
from my_micrograd.nn import MLP

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

model = MLP(2, [4, 4, 1])

learning_rate = 0.005
epochs = 10

for epoch in range(epochs):
    train_df = train_df.sample(frac=1)

    total_loss = 0.0

    for _, row in train_df.iterrows():
        x = [Value(row.Shots_Scaled), Value(row.Movement)]

        pred = model(x)
        target = Value(row.KD_Spread_Scaled)
        loss = (pred-target)**2

        total_loss += loss.data

        model.zero_grad()
        loss.backward()

        for p in model.parameters():
            p.data -= learning_rate * p.grad

    print(f"Epoch {epoch}, Loss: {total_loss / len(train_df)}")

Epoch 0, Loss: 1.001617923719916
Epoch 1, Loss: 0.9436573973314651
Epoch 2, Loss: 0.9285466612594184
Epoch 3, Loss: 0.9325351800772865
Epoch 4, Loss: 0.9237504230797315
Epoch 5, Loss: 0.929667907418132
Epoch 6, Loss: 0.9217367884261881
Epoch 7, Loss: 0.9255053496247948
Epoch 8, Loss: 0.9218251022481189
Epoch 9, Loss: 0.9213115555302713


In [3]:
def evaluate(df):
    errors = []
    for _, row in df.iterrows():
        x = [Value(row.Shots_Scaled), Value(row.Movement)]
        pred = model(x).data
        errors.append((pred-row.KD_Spread_Scaled)**2)
    return sum(errors)/len(errors)

print('Test MSE: ', evaluate(test_df))

Test MSE:  0.7226055242157511


In [4]:
kd_spread_mean = df['KD_Spread'].mean()
kd_spread_std = df['KD_Spread'].std()

def unscale_kd_spread(x):
    return x * kd_spread_std + kd_spread_mean

def predict(shots, movement):
    x = [Value(shots / df['Shots'].max()), Value(movement)]
    pred = model(x).data
    return unscale_kd_spread(pred)

print(predict(300, 0.8))

-1.2852673102820082


In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a grid of values
shots_vals = np.linspace(0, df["Shots"].max(), 50)
movement_vals = np.linspace(0, 1, 50)

S, M = np.meshgrid(shots_vals, movement_vals)

# Compute predictions for each point on the grid
Z = np.zeros_like(S)

for i in range(S.shape[0]):
    for j in range(S.shape[1]):
        shots_scaled = S[i, j] / df["Shots"].max()
        movement_scaled = M[i, j]
        x = [Value(shots_scaled), Value(movement_scaled)]
        pred_scaled = model(x).data
        Z[i, j] = unscale_kd_spread(pred_scaled)

# Plot the surface
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(S, M, Z, cmap='viridis', alpha=0.7, edgecolor='none')

# Scatter overlay of actual data
ax.scatter(
    df["Shots"],
    df["Movement"],
    df["KD_Spread"],
    color='red',
    s=20,
    label='Actual Data'
)

ax.set_xlabel("Shots")
ax.set_ylabel("Movement")
ax.set_zlabel("KD Spread")
ax.set_title("MLP Prediction Surface with Actual Data Overlay")
ax.legend()

plt.show()